In [100]:
import os
import pickle
import torch
import pandas as pd

from tqdm import tqdm
from sklearn.metrics import f1_score
from collections import Counter

text_counter = Counter()
tqdm.pandas()
mps_device = torch.device("mps")
torch.cuda.empty_cache()

In [101]:
WEIGHTS_PATH = './../weights/'
DATASET_PATH = './trainset.csv'
INTERMEDIATE_PATH = './../intermediate/'
RESULTS_PATH = './../results/'
VALIDATION = './../data/Validation.txt'


embedding_size = 512
no_of_rows     = None
window_size    = 3
max_epochs     = 10000
learning_rate  = 0.001
reg_lambda     = 1
batch_size     = 8192
train_split    = 0.8
is_train       = True
new_weights    = True
use_biases     = False
problem        = 'skipgram'

epochs_stat = []

In [102]:
class Net(object):
    
    def __init__(self):
        self.y_pred = None
        self.emb    = None

        if(os.path.exists(WEIGHTS_PATH + 'skipgram_weights_' + str(vocab_size) + '.pt') and (new_weights == False)):
            self.weights = torch.load(WEIGHTS_PATH + 'skipgram_weights_' + str(vocab_size) + '.pt')
            self.biases = torch.load(WEIGHTS_PATH + 'skipgram_biases_' + str(vocab_size) + '.pt')
        else:
            self.weights = []
            self.biases  = []
            
            self.weights.append(torch.rand((vocab_size, embedding_size), device=mps_device) * 2 - 1)
            self.weights.append(torch.rand((embedding_size, vocab_size), device=mps_device) * 2 - 1)
            if(use_biases):
                self.biases.append(torch.rand((embedding_size), device=mps_device) * 2 - 1)
                self.biases.append(torch.rand((vocab_size), device=mps_device) * 2 - 1)
            else:
                self.biases.append(torch.zeros((embedding_size), device=mps_device))
                self.biases.append(torch.zeros((vocab_size), device=mps_device))

    def __call__(self, X):
        self.emb = (torch.matmul(X,self.weights[0]) + self.biases[0])
        self.y_pred = torch.softmax(torch.matmul(self.emb, self.weights[1]) + self.biases[1], dim=1)
        return self.y_pred

    def backward(self, X, y, lamda):
        
        del_W = []
        del_b = []

        delta = self.y_pred - y
        del_b.insert(0,torch.sum(delta, axis = 0, keepdims = True))
        del_W.insert(0,torch.matmul(self.emb.T, delta) + lamda * (self.weights[1]))

        delta = torch.matmul(delta, self.weights[1].T) * (self.emb)
        del_b.insert(0,torch.sum(delta, axis = 0, keepdims = True))
        del_W.insert(0,torch.matmul(X.T, delta) + lamda * (self.weights[0]))
        return del_W, del_b

    
class Optimizer(object):
    '''
    '''
    def __init__(self, learning_rate, weights, biases, optimizer="gradient"):
        
        
        self.optimizer = optimizer
        
        self.m_dw = [torch.zeros((w.shape), device=mps_device) for w in weights]
        self.m_db = [torch.zeros((b.shape), device=mps_device) for b in biases]
        self.v_dw = [torch.zeros((w.shape), device=mps_device) for w in weights]
        self.v_db = [torch.zeros((b.shape), device=mps_device) for b in biases]
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8
        self.eta = learning_rate
        self.t = 0

    def step(self, weights, biases, delta_weights, delta_biases):
        
        if(self.optimizer == "gradient"):
            return self.gradient(weights, biases, delta_weights, delta_biases)
        elif(self.optimizer == "adam"):
            return self.adam(weights, biases, delta_weights, delta_biases)
    
    def adam(self, weights, biases, delta_weights, delta_biases):
        self.t += 1

        self.m_dw = [self.beta1 * m + (1 - self.beta1) * del_w for m, del_w in zip(self.m_dw, delta_weights)]
        self.m_db = [self.beta1 * m + (1 - self.beta1) * del_b for m, del_b in zip(self.m_db, delta_biases)]
        self.v_dw = [self.beta2 * v + (1 - self.beta2) * (del_w**2) for v, del_w in zip(self.v_dw, delta_weights)]
        self.v_db = [self.beta2 * v + (1 - self.beta2) * (del_b**2) for v, del_b in zip(self.v_db, delta_biases)]

        # bias correction
        m_hat_dw = [m / (1 - self.beta1 ** self.t) for m in self.m_dw]
        v_hat_dw = [v / (1 - self.beta2 ** self.t) for v in self.v_dw]

        m_hat_db = [m / (1 - self.beta1 ** self.t) for m in self.m_db]
        v_hat_db = [v / (1 - self.beta2 ** self.t) for v in self.v_db]

        # update weights and biases
        weights = [w - self.eta * m_hat / ((torch.sqrt(v_hat) + self.epsilon)) for w, m_hat, v_hat in zip(weights, m_hat_dw, v_hat_dw)] 
        biases = [b - self.eta * m_hat / ((torch.sqrt(v_hat) + self.epsilon)) for b, m_hat, v_hat in zip(biases, m_hat_db, v_hat_db)]
        return weights, biases

    
    def gradient(self, weights, biases, delta_weights, delta_biases):
        for i in range(len(weights)):
            weights[i] = weights[i] - self.eta*delta_weights[i]
            biases[i] = biases[i] - self.eta*delta_biases[i]
        return weights, biases

In [103]:
def get_embeddings(word):
    try:
        val = word2idx[word]
        input_arr = torch.zeros((vocab_size), dtype=torch.float, device=mps_device)
        input_arr[val] = 1
        emb = torch.matmul(input_arr,net.weights[0]) + net.biases[0]
        return emb
    except:
        return torch.empty(0, dtype=torch.float, device=mps_device)

def get_result(word1, word2, word3, problem='skipgram', window=4):
    w1_emb = get_embeddings(word1)
    w2_emb = get_embeddings(word2)    
    w3_emb = get_embeddings(word3)    
    if((torch.numel(w1_emb) != 0) and (torch.numel(w2_emb) != 0) and (torch.numel(w3_emb) != 0)):
        w4_emb = w1_emb + w3_emb - w2_emb
        output = torch.softmax(torch.matmul(w4_emb, net.weights[1]) + net.biases[1], dim=1)
        if(problem=='skipgram'):
            topk_preds = torch.topk(output, k=window).indices.tolist()
            ans = [idx2word[x] for x in topk_preds[0]]
        else:
            ans = idx2word[torch.argmax(output).item()]
        return ans
    return "UNK"


def get_embeddings(word):
    try:
        val = word2idx[word]
        input_arr = torch.zeros((vocab_size), dtype=torch.float, device=mps_device)
        input_arr[val] = 1
        emb = torch.matmul(input_arr,net.weights[0]) + net.biases[0]
        return emb
    except:
        return torch.empty(0, dtype=torch.float, device=mps_device)
    
def get_accuracy(validation):
    accuracy = torch.empty(0, dtype=torch.float, device=mps_device)
    for _, row in validation.iterrows():
        w1_emb = get_embeddings(row['word1'])
        w2_emb = get_embeddings(row['word2'])    
        w3_emb = get_embeddings(row['word3'])
        w4_emb = get_embeddings(row['word4'])
        
        if((torch.numel(w1_emb) != 0) and (torch.numel(w2_emb) != 0) and (torch.numel(w3_emb) != 0)):
            pred_emb = w1_emb + w3_emb - w2_emb
            accuracy = torch.cat(accuracy, cosine_similarity(pred_emb, w4_emb), dim=0)
    return torch.mean(accuracy, dim=0)

def cosine_similarity(A, B):
    C = torch.squeeze(A)
    D = torch.squeeze(B)
    return torch.dot(C, D) / (torch.norm(C) * torch.norm(D))

def cross_entropy_loss(y_pred, y_true):
    if(mps_device.type == 'mps'):
        return -torch.mean(custom_nansum(y_true * torch.log(y_pred), axis=-1))
    else:
        return -torch.mean(torch.nansum(y_true * torch.log(y_pred), axis=-1))

def get_dataset(words, window_size):
    data = pd.DataFrame(columns=["word", "context_words"])
    for index, word in enumerate(words):
        context_words = words[max(0, index - window_size): index] + words[index + 1: index + window_size + 1]
        context_words = list(set(context_words))
        data.loc[len(data.index)] = [word, context_words]
    global dataset
    dataset = pd.concat([dataset,data])
    
def custom_nansum(input_tensor, axis=None):
    # Replace NaN values with 0
    input_tensor = torch.where(torch.isnan(input_tensor), torch.tensor(0., device=input_tensor.device), input_tensor)
    
    # Compute the sum over the specified axis
    if axis is None:
        return torch.sum(input_tensor)
    else:
        return torch.sum(input_tensor, dim=axis)
    
    
def get_batch_data(start_index, end_index):

    batch_dataset = dataset[start_index:end_index].reset_index(drop=True)
    batch_size = batch_dataset.shape[0]
    batch_input = torch.zeros([batch_size, vocab_size], dtype=torch.float, device=mps_device)
    batch_output = torch.zeros([batch_size, vocab_size], dtype=torch.float, device=mps_device)

    for index, data in batch_dataset.iterrows():
        batch_input[index, data['word']] = 1
        for ind in data['context_words']:
            batch_output[index, ind] = 1
            
    return batch_input, batch_output


def train(net, optimizer, lamda, max_epochs, dev_input, dev_target, batch_size, train_size):

    stop = False
    value = 999999999
    
    for e in range(max_epochs):
        first_loss = True
        epoch = {}
        batches = []
        losses = []
        for start_index in range(0, train_size, batch_size):
            end_index = min(start_index + batch_size, train_size)
            batch_input, batch_target = get_batch_data(start_index, end_index)
            pred = net(batch_input)

            # Compute gradients of loss w.r.t. weights and biases
            dW, db = net.backward(batch_input, batch_target, lamda)

            # Get updated weights based on current weights and gradients
            weights_updated, biases_updated = optimizer.step(net.weights, net.biases, dW, db)

            # Update model's weights and biases
            net.weights = weights_updated
            net.biases = biases_updated
            loss = cross_entropy_loss(pred, batch_target)
            print(e, start_index, loss.item())
            batches.append(start_index)
            losses.append(loss.item())
            if(torch.isnan(loss) or torch.isinf(loss)):
                stop = True
                break

            if(first_loss):
                first_loss = False
                if(value<loss.item()):
                    stop = True
                    break
                else:
                    value = loss.item()

        epoch['batches'] = batches
        epoch['losses'] = losses
            

        dev_pred = net(dev_input)
        indices = torch.topk(dev_pred, k=window_size, dim=1)[1][:, -window_size:]
        converted_matrix = torch.zeros_like(dev_pred)
        converted_matrix[torch.arange(dev_pred.shape[0])[:, None], indices] = 1
        numpy_dev_target = dev_target.cpu().numpy()
        converted_matrix = converted_matrix.cpu().numpy()
        score = f1_score(numpy_dev_target, converted_matrix, average='micro')
        print('F1 Score on dev data: {:.5f}'.format(score))
        epoch['f1_score'] = score
        epochs_stat.append(epoch)
        if(stop):
            break

### Loading Data

In [104]:
df = pd.read_csv(DATASET_PATH ,nrows = no_of_rows)
df = df.dropna()

df['sentences'] = df['sentences'].apply(lambda sentence : sentence.split())
_ = df['sentences'].apply(text_counter.update)
text_counter.update(['UNK'])
vocab_size = len(text_counter)

words, _ = zip(*text_counter.most_common(vocab_size))
word2idx = {w: i for i, w in enumerate(words)}
idx2word = {i: w for i, w in enumerate(words)}


# dataset = pd.read_pickle('dataset.pkl')
df['sentences'] = df['sentences'].apply(lambda words : [word2idx[word] for word in words])

# if(no_of_rows==None and os.path.exists(INTERMEDIATE_PATH + 'dataset.pkl')):
#    dataset = pd.read_pickle(INTERMEDIATE_PATH + 'dataset.pkl') 

# elif(os.path.exists(INTERMEDIATE_PATH + 'dataset_'+str(no_of_rows)+'.pkl')):
#     dataset = pd.read_pickle(INTERMEDIATE_PATH + 'dataset_'+str(no_of_rows)+'.pkl')
# else:
dataset = pd.DataFrame(columns=["word", "context_words"])
_ = df['sentences'].progress_apply(lambda x : get_dataset(x, window_size))

dataset = dataset.sample(frac=1).reset_index(drop=True)
if(no_of_rows==None):
    dataset.to_pickle(INTERMEDIATE_PATH + 'dataset.pkl') 
else:
    dataset.to_pickle(INTERMEDIATE_PATH + 'dataset_'+str(no_of_rows)+'.pkl')

data_size = dataset.shape[0]

print("Vocab length :", vocab_size)
print("Dataset size :", data_size)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 425/425 [00:00<00:00, 777.19it/s]

Vocab length : 5009
Dataset size : 1223334


### Training the model

In [105]:
train_size = max(int(train_split*data_size), data_size-5000)

net = Net()
optimizer = Optimizer(learning_rate, net.weights, net.biases, optimizer="adam")

In [106]:
if(is_train):
    dev_input, dev_target = get_batch_data(train_size, data_size)
    train(net, optimizer, reg_lambda, max_epochs, dev_input, dev_target, batch_size, train_size)

    torch.save(net.weights, WEIGHTS_PATH + 'skipgram_weights_' + str(vocab_size) + '.pt')
    torch.save(net.biases, WEIGHTS_PATH + 'skipgram_biases_' + str(vocab_size) + '.pt')
else:
    net.weights = torch.load(WEIGHTS_PATH + 'skipgram_weights_' + str(vocab_size) + '.pt')
    net.biases = torch.load(WEIGHTS_PATH + 'skipgram_biases_' + str(vocab_size) + '.pt')

0 0 47.576759338378906
F1 Score on dev data: 0.00627
1 0 47.09143829345703
F1 Score on dev data: 0.00627
2 0 46.61103820800781
F1 Score on dev data: 0.00627
3 0 46.13486862182617
F1 Score on dev data: 0.00627
4 0 45.663150787353516
F1 Score on dev data: 0.00627
5 0 45.196102142333984
F1 Score on dev data: 0.00627
6 0 44.733863830566406
F1 Score on dev data: 0.00627
7 0 44.27666473388672
F1 Score on dev data: 0.00627
8 0 43.82476806640625
F1 Score on dev data: 0.00627
9 0 43.37847900390625
F1 Score on dev data: 0.00627
10 0 42.93806076049805
F1 Score on dev data: 0.00627
11 0 42.50373077392578
F1 Score on dev data: 0.00627
12 0 42.07563781738281
F1 Score on dev data: 0.00627
13 0 41.65381622314453
F1 Score on dev data: 0.00627
14 0 41.23833084106445
F1 Score on dev data: 0.00627
15 0 40.82908630371094
F1 Score on dev data: 0.00627
16 0 40.42593765258789
F1 Score on dev data: 0.00627
17 0 40.02873992919922
F1 Score on dev data: 0.00627
18 0 39.63747024536133
F1 Score on dev data: 0.00627

153 0 19.990949630737305
F1 Score on dev data: 0.00471
154 0 19.919055938720703
F1 Score on dev data: 0.00471
155 0 19.84691047668457
F1 Score on dev data: 0.00471
156 0 19.77449607849121
F1 Score on dev data: 0.00471
157 0 19.701801300048828
F1 Score on dev data: 0.00471
158 0 19.62881088256836
F1 Score on dev data: 0.00471
159 0 19.555509567260742
F1 Score on dev data: 0.00471
160 0 19.481889724731445
F1 Score on dev data: 0.00471
161 0 19.40793228149414
F1 Score on dev data: 0.00471
162 0 19.33363151550293
F1 Score on dev data: 0.00471
163 0 19.25897216796875
F1 Score on dev data: 0.00471
164 0 19.18393898010254
F1 Score on dev data: 0.00471
165 0 19.1085262298584
F1 Score on dev data: 0.00471
166 0 19.03272247314453
F1 Score on dev data: 0.00471
167 0 18.956506729125977
F1 Score on dev data: 0.00471
168 0 18.8798770904541
F1 Score on dev data: 0.00471
169 0 18.80281639099121
F1 Score on dev data: 0.00471
170 0 18.725313186645508
F1 Score on dev data: 0.00471
171 0 18.64735794067382

305 0 7.19760799407959
F1 Score on dev data: 0.09098
306 0 7.163403034210205
F1 Score on dev data: 0.09412
307 0 7.129883289337158
F1 Score on dev data: 0.09412
308 0 7.0970234870910645
F1 Score on dev data: 0.09569
309 0 7.064802169799805
F1 Score on dev data: 0.09725
310 0 7.03319787979126
F1 Score on dev data: 0.09725
311 0 7.002192974090576
F1 Score on dev data: 0.09882
312 0 6.97177267074585
F1 Score on dev data: 0.10039
313 0 6.941921710968018
F1 Score on dev data: 0.10196
314 0 6.912627696990967
F1 Score on dev data: 0.10196
315 0 6.883881568908691
F1 Score on dev data: 0.10196
316 0 6.855671405792236
F1 Score on dev data: 0.10196
317 0 6.827991485595703
F1 Score on dev data: 0.10353
318 0 6.800830364227295
F1 Score on dev data: 0.10353
319 0 6.774185657501221
F1 Score on dev data: 0.10667
320 0 6.748048305511475
F1 Score on dev data: 0.10667
321 0 6.722413539886475
F1 Score on dev data: 0.10667
322 0 6.697274684906006
F1 Score on dev data: 0.10824
323 0 6.672626495361328
F1 Sco

### Validation

In [107]:
validation = pd.read_csv(VALIDATION, sep=' ', names=['word1','word2','word3','word4'])

validation['word1'] = validation['word1'].apply(lambda x : x.lower())
validation['word2'] = validation['word2'].apply(lambda x : x.lower())
validation['word3'] = validation['word3'].apply(lambda x : x.lower())
validation['word4'] = validation['word4'].apply(lambda x : x.lower())

validation['pred'] = validation[['word1','word2','word3']].progress_apply(lambda x : get_result(x['word1'], x['word2'], x['word3'], problem=problem, window=10), axis = 1)
validation['found'] = validation[['word4','pred']].progress_apply(lambda x : x['word4'] in x['pred'], axis=1)

# count = len(os.listdir(RESULTS_PATH))
# validation.to_csv(RESULTS_PATH + "value_"+str(count)+'_'+str(vocab_size)+'_'+str(no_of_rows)+problem+'.csv',index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 991/991 [00:00<00:00, 115620.45it/s]


In [108]:
validation[validation['found']==True]

,word1,word2,word3,word4,pred,found
0,walk,walks,see,sees,"[is, sees, walking, wife, japan, lower, mango,...",True
1,walk,walks,shuffle,shuffles,"[is, shuffles, Bangkok, wife, shuffling, spend...",True
2,walk,walks,sing,sings,"[is, sings, Senegal, singing, wife, Guinea, bi...",True
3,walk,walks,sit,sits,"[is, sitting, walking, shuffles, sits, wife, b...",True
5,walk,walks,speak,speaks,"[is, walking, harder, wife, think, spending, s...",True
...,...,...,...,...,...,...
983,algeria,dinar,iran,rial,"[is, dinar, israeli, rial, possibly, India, de...",True
985,argentina,peso,usa,dollar,"[is, peso, dollar, stepmother, France, finds, ...",True
986,argentina,peso,nigeria,naira,"[is, peso, naira, finds, unethical, warm, Fran...",True
987,argentina,peso,iran,rial,"[is, peso, rial, France, cool, Norwegian, swed...",True


In [43]:
x = validation[validation['found']==False].reset_index(drop=True)

In [27]:
x = x[['word1','word2','word3','word4']]
x.to_csv("addition.csv",index=False)

### Example

In [114]:


get_result('run','ran','play', problem='skipgram', window=5)

# get_similarity('queen', 'woman')

['plays', 'is', 'flying', 'certain', 'groom']

### Plots

In [ ]:
epochs = list(range(len(epochs_stat)))
total_loss = []
for i in range(len(epochs_stat)):
    total_loss.append(sum(epochs_stat[i]['losses'])/len(epochs_stat[i]['losses']))
    
plot_loss_vs_batch(epochs, total_loss)

In [ ]:
plot_f1_vs_epoch(epochs,x)

In [ ]:
import matplotlib.pyplot as plt

def plot_f1_vs_epoch(batches, losses):
    plt.figure()
    plt.plot(batches, losses)
    plt.xlabel('Epochs')
    plt.ylabel('F1 score')
    plt.title('Fl score vs. Epochs on Dev data')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_vs_batch(batches, losses):
    plt.figure()
    plt.plot(batches, losses)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss vs. Epochs')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the input list
input_list = ['30k', '123k', '216k', '391k', '1579k']

# Define the two lists to compare
list1 = [35, 154, 387, 0, 0]
list2 = [7, 19, 35, 68, 152]

# Set the width of the bars
barWidth = 0.35

# Set the positions of the bars on the x-axis
r1 = range(len(input_list))
r2 = [x + barWidth for x in r1]

# Create the plot
plt.bar(r1, list1, color='red', width=barWidth, edgecolor='white', label='time with np arrays(in min)')
plt.bar(r2, list2, color='blue', width=barWidth, edgecolor='white', label='time with tensors(in min)')

# Add xticks on the middle of the group bars
plt.xlabel('Sentences')
plt.ylabel('Time for 10 epochs')
plt.xticks([r + barWidth / 2 for r in r1], input_list)

# Create a legend & show the plot
plt.legend()
plt.show()

